In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as skmts
import scipy.stats as scistat
import glob
plt.style.use('fivethirtyeight')

In [2]:
def eval_mts(y_list, y_pred_list, task_str):
    """return evaluation  metrices"""
    mask = ~np.isnan(y_pred_list)
    y_list = y_list[mask]
    y_pred_list = y_pred_list[mask]
    if task_str == 'classification':
        auc = skmts.roc_auc_score(y_list, y_pred_list, average='micro')
        aucprc = skmts.average_precision_score(y_list, y_pred_list, average='micro')
        y_pred_list = (y_pred_list>0.5).astype(int)
        acc = skmts.accuracy_score(y_list, y_pred_list)
        mcc = skmts.matthews_corrcoef(y_list, y_pred_list)
        f1 = skmts.f1_score(y_list, y_pred_list, average='micro')
        precision = skmts.precision_score(y_list, y_pred_list)
        recall = skmts.recall_score(y_list, y_pred_list)
        kappa = skmts.cohen_kappa_score(y_list, y_pred_list)
        balanced_acc = skmts.balanced_accuracy_score(y_list, y_pred_list)

        df = pd.DataFrame({'metrices': ['AUC', 'AUPRC', 'Accuracy', 'MCC', 'F1', 'Precision', 'Recall', 'Kappa', 'Balanced_Accuracy'],
                           'score': [auc, aucprc, acc, mcc, f1, precision, recall, kappa, balanced_acc]})

        tn, fp, fn, tp = skmts.confusion_matrix(y_list, y_pred_list).ravel()
        print(f'confusion matrix: TN={tn}, FP={fp}, FN={fn}, TP={tp}')

    elif task_str == 'regression':
        mae = skmts.mean_absolute_error(y_list, y_pred_list)
        mse = skmts.mean_squared_error(y_list, y_pred_list)
        rmse = skmts.mean_squared_error(y_list, y_pred_list, squared=False)
        r2 = skmts.r2_score(y_list, y_pred_list)
        pcc, pval = scistat.pearsonr(y_list, y_pred_list)
        spr, pval = scistat.spearmanr(y_list, y_pred_list)

        df = pd.DataFrame({'metrices': ['MAE', 'MSE', 'RMSE', 'R2', 'Pearson r', 'Spearman r'],
                           'score': [mae, mse, rmse, r2, pcc, spr]})
    else:
        raise ValueError(f'Error, {task_str} should be either classification or regression!!!')
    return df

In [3]:
def get_overall_performance(data_path):
    """return overall performance across folds
    :param data_path: path to evaluation_metrices.txt
    """
    # collect files
    df_list = []
    for f in glob.glob(data_path+'*.evaluation_metrices.txt'):
        df = pd.read_csv(f, header=0, sep="\t")
        #print(df)
        df_list.append(df.set_index(['metrics']))
    df = pd.concat(df_list, axis=1)
    
    # calculate mean , standard deviation
    df = pd.concat([df.mean(axis=1).to_frame(name='mean'), df.std(axis=1).to_frame(name='standard deviation')], axis=1)
    
    # display message
    print(f'Overall performance across {len(df_list)} folds:\n{df}')
    return df

In [4]:
def get_perTriplet_performance(data_path):
    """return overall performance across folds
    :param data_path: path to model_prediction.txt
    """
    # collect files
    df_list = []
    for f in glob.glob(data_path+'*.model_prediction.txt'):
        df = pd.read_csv(f, header=0, sep="\t")
        #df.columns = ['synergy_label', 'predicted_synergy', 'icombo']
        #print(df)
        df_list.append(df)
    df = pd.concat(df_list, axis=0)
    
    # get triplet from icombo
    df[['did_row', 'did_col', 'cid', 'conc_r', 'conc_c']]= df['icombo'].str.split('=', expand=True)
    df['triplet'] = df['did_row'] + '_' + df['did_col'] + '_' + df['cid']
   
    
    # collect qualified triplets: have positives and negatives  
    g = df.groupby(by=['triplet'])['synergy_label'].agg('mean').to_frame('#label')
    all_pos = g[g['#label']==1].index.tolist()
    all_neg = g[g['#label']==0].index.tolist()
    triplet_list = sorted(set(g.index) - set(all_pos+all_neg))
    
    # get per-triplet performance
    record_list = [] # (N, AUC, AUPRC)
    grps = df.groupby(['triplet'])
    for triplet in triplet_list:
        sample = grps.get_group(triplet)
        auc = skmts.roc_auc_score(sample['synergy_label'], sample['predicted_synergy'])
        auprc = skmts.average_precision_score(sample['synergy_label'], sample['predicted_synergy'])
        record_list.append( (len(sample), auc, auprc) )
    df = pd.DataFrame.from_records(record_list, index=triplet_list, columns=['N', 'AUC', 'AUPRC'])
    df = df.sort_values(by=['N', 'AUPRC', 'AUC'], ascending=False)
    print(f'Top 10 triplet:\n{df.head(10)}')
    
    # calculate mean , standard deviation
    print(f'per-triplet performance')
    print(pd.concat([df.mean(axis=0).to_frame(name='mean'), df.std(axis=0).to_frame(name='standard deviation')], axis=1))
    
    return df

In [5]:
def get_perRegion_performance(data_path):
    """return overall performance across folds
    :param data_path: path to model_prediction.txt
    """
    # collect files
    df_list = []
    for f in glob.glob(data_path+'*.model_prediction.txt'):
        df = pd.read_csv(f, header=0, sep="\t")
        df_list.append(df)
    df = pd.concat(df_list, axis=0)
    
    # get per-region performance
    record_list = [] # (N, AUC, AUPRC)
    region_list = sorted(df['region_label'].unique())
    grps = df.groupby(['region_label'])
    for region in region_list:
        sample = grps.get_group(region)
        auc = skmts.roc_auc_score(sample['synergy_label'], sample['predicted_synergy'])
        auprc = skmts.average_precision_score(sample['synergy_label'], sample['predicted_synergy'])
        record_list.append( (len(sample), auc, auprc) )
    df = pd.DataFrame.from_records(record_list, index=region_list, columns=['N', 'AUC', 'AUPRC'])
    df = df.sort_values(by=['N', 'AUPRC', 'AUC'], ascending=False)
    
    # calculate mean , standard deviation
    print(f'per-region performance:\n{df}')
    
    return df

In [8]:
data_path = '../classification/random_split/EXP/c3/DRUG_CELL_DOSE/transform_ssGSEA/kmer9/'
get_overall_performance(data_path)
get_perTriplet_performance(data_path)

Overall performance across 10 folds:
                       mean  standard deviation
metrics                                        
AUC                0.773939            0.009924
AUPRC              0.900054            0.006355
Accuracy           0.691096            0.011161
MCC                0.359916            0.010806
F1                 0.691096            0.011161
Precision          0.874563            0.007308
Recall             0.678867            0.022359
Kappa              0.336354            0.011991
Balanced_Accuracy  0.702191            0.005942
Top 10 triplet:
                           N       AUC     AUPRC
did_1730_did_206_cid_122  27  0.884615  0.995558
did_1730_did_206_cid_6    24  0.521739  0.974095
did_1730_did_206_cid_9    22  0.947368  0.992823
did_1730_did_966_cid_122  20  0.947368  0.997368
did_1730_did_206_cid_132  18  0.882353  0.993272
did_1117_did_180_cid_123  18  0.987500  0.990909
did_908_did_657_cid_123   18  0.928571  0.981548
did_1815_did_206_cid_65   1

,N,AUC,AUPRC
did_1730_did_206_cid_122,27,0.884615,0.995558
did_1730_did_206_cid_6,24,0.521739,0.974095
did_1730_did_206_cid_9,22,0.947368,0.992823
did_1730_did_966_cid_122,20,0.947368,0.997368
did_1730_did_206_cid_132,18,0.882353,0.993272
...,...,...,...
did_968_did_180_cid_125,2,0.000000,0.500000
did_972_did_1117_cid_127,2,0.000000,0.500000
did_972_did_968_cid_118,2,0.000000,0.500000
did_972_did_968_cid_122,2,0.000000,0.500000
